In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc

# Créer une session Spark
spark = SparkSession.builder \
    .appName("Airbnb Expensive Trips") \
    .getOrCreate()

df = spark.read.parquet("hdfs://namenode:9000/data/data_trans/")


df = df.select("origin_id", "destination_id", "price_cents")

df = df.na.drop(subset=["price_cents"])

rdd = df.rdd.map(lambda row: ((row["origin_id"], row["destination_id"]), row["price_cents"]))

max_prices = rdd.reduceByKey(lambda a, b: max(a, b))

max_prices_df = max_prices.map(lambda x: (x[0][0], x[0][1], x[1])) \
                          .toDF(["origin_id", "destination_id", "max_price_cents"])

max_prices_df = max_prices_df.sort(desc("max_price_cents"))

max_prices_df.show(10)

